In [1]:
from datetime import timedelta, date as dt


from dateutil.relativedelta import relativedelta

import math
import numpy as np
import pandas as pd

import matplotlib.ticker as ticker
import matplotlib.cm as cm
import matplotlib as mpl
from matplotlib.ticker import FuncFormatter
from matplotlib.gridspec import GridSpec
import matplotlib.pyplot as plt


def get_how_many_2_weeks_period(start_dt, end_dt):
    """
    Computes the number of 2 weeks periods between 2 dates
    :param start_dt: Beginning of period
    :param end_dt: end of period
    :return int: The number of 2 week periods between 2 dates
    """
    monday1 = (start_dt - timedelta(days=start_dt.weekday()))
    monday2 = (end_dt - timedelta(days=end_dt.weekday()))
    return (monday2 - monday1).days / 14


def get_how_many_month_period(start_dt, end_dt):
    """
    :param start_dt:
    :param end_dt:
    :return:
    """
    return relativedelta(end_dt, start_dt).years * 12 + relativedelta(end_dt, start_dt).months


def years_ago(years, from_date=None):
    if from_date is None:
        from_date = dt.today()
    try:
        return from_date.replace(year=from_date.year - years)
    except ValueError:
        # Must be 2/29!
        assert from_date.month == 2 and from_date.day == 29  # can be removed
        return from_date.replace(month=2, day=28,
                                 year=from_date.year - years)


def num_years(begin, end=None):
    if end is None:
        end = dt.today()
    if type(end) == str:
        end = date_from_string_to_dt(end)

    if type(begin) == str:
        begin = date_from_string_to_dt(begin)
    #     print(begin)
    num_years = int((end - begin).days / 365.25)
    if begin > years_ago(num_years, end):
        return num_years - 1
    else:
        return num_years


def date_from_string_to_dt(date_str):
    assert type(date_str) == str, "date_str type must be str and date_str is " + str(type(date_str))
    dt_list = date_str.split("-")
    assert len(dt_list) == 3, "date_str type must be in the format 'xxxx'-'xx'-'xx' "
    for elem in dt_list:
        int(elem)
        # print(int(elem))
    return dt(int(dt_list[0]), int(dt_list[1]), int(dt_list[2]))


class FinancialFunctions(object):
    # to do: interest rate = 0?
    def __init__(self, amortization_periods_in_years, compounding_period_per_year, interest_rate_annual_nominal_rate,
                 principal_pv=None, pmt=None, **kwargs):

        """

        :param float amortization_periods_in_years: Number of years over which the loan will be repaid
        :param int compounding_period_per_year: Number of time the interest will be compounded by lender, typically 26, 12, or 1 time
        :param float interest_rate_annual_nominal_rate: Nominal interest rate
        :param float principal_pv: Present value of borrowed amount (Loan VS investment) if not None, will determine the payment
        :param float pmt: pmt per interest compounding period
        :param str kwargs: amortisation_period_start_date_dt: if specified, will create a calendar
        :param int kwargs: : if specified, will create a calendar
        """
        self.interest_rate_annual_nominal_rate = interest_rate_annual_nominal_rate
        self.interest_rate_compounding_period_per_year = compounding_period_per_year
        self.amortisation_period_total_number_of_years = amortization_periods_in_years

        self.amortisation_total_number_of_interest_compounding_periods = amortization_periods_in_years * compounding_period_per_year
        self.interest_rate_effective_rate_per_period = interest_rate_annual_nominal_rate / compounding_period_per_year

        self.accumulation_factor = ((1 + self.interest_rate_effective_rate_per_period) ** (
            self.amortisation_total_number_of_interest_compounding_periods) - 1) / self.interest_rate_effective_rate_per_period
        self.actualisation_factor = (1 - (1 / (
                    1 + self.interest_rate_effective_rate_per_period)) ** self.amortisation_total_number_of_interest_compounding_periods) / self.interest_rate_effective_rate_per_period

        self.prepayment_amount = None
        self.prepayment_time = None
        self.prepayment_period_number = None
        self.new_amortisation_schedule = None

        if 'start_date' in kwargs:
            self.amortisation_period_start_date_dt = date_from_string_to_dt(kwargs['start_date'])
            self.amortisation_period_end_date_dt = self.amortisation_period_start_date_dt + timedelta(
                weeks=52 * self.amortisation_period_total_number_of_years)
            ###
        else:
            self.amortisation_period_start_date_dt = None
            self.pmt_calendar = None

        if pmt is None:
            assert principal_pv is not None, 'PMT is None, principal cant be None'
            self.principal_pv = principal_pv
            self.pmt = principal_pv / self.actualisation_factor
            self.principal_fv = self.principal_pv * (
                        1 + self.interest_rate_effective_rate_per_period) ** self.amortisation_total_number_of_interest_compounding_periods
            assert self.pmt * self.accumulation_factor - self.principal_fv < 1, "Future Value are not consistent " + " " + str(
                self.pmt * self.accumulation_factor - self.principale_fv)

        if principal_pv is None:
            assert pmt is not None, 'Principal is None, pmt cant be None'
            self.pmt = pmt
            self.principal_pv = self.pmt * self.actualisation_factor
            self.principal_fv = self.pmt * self.accumulation_factor
            assert self.principal_pv * (
                        1 + self.interest_rate_effective_rate_per_period) ** self.amortisation_total_number_of_interest_compounding_periods - self.principal_fv < 1, "Future Value are not consistent " + " " + str(
                self.principal_pv * (
                            1 + self.interest_rate_effective_rate_per_period) ** self.amortisation_total_number_of_interest_compounding_periods - self.principal_fv)

        self.principal_nominal_value = self.amortisation_total_number_of_interest_compounding_periods * self.pmt
        self.total_interest_paid = self.principal_fv - self.principal_nominal_value

    def get_start_date(self):
        return self.amortisation_period_start_date_dt

    def get_accumulation_factor_for(self, number_of_compounding_period):
        return ((1 + self.interest_rate_effective_rate_per_period) ** (
            number_of_compounding_period) - 1) / self.interest_rate_effective_rate_per_period

    def get_actualisation_factor(self, effective_interest_rate_per_period=None, number_of_compounding_periods=None):
        if effective_interest_rate_per_period is None:
            effective_interest_rate_per_period = self.interest_rate_effective_rate_per_period
        if number_of_compounding_periods is None:
            number_of_compounding_periods = self.amortisation_total_number_of_interest_compounding_periods
        return (1 - (1 / (
                    1 + effective_interest_rate_per_period)) ** number_of_compounding_periods) / effective_interest_rate_per_period

    def get_number_of_period_interest_rate_compounding_between_2_dates(self, start_dt, end_dt):
        """
            Computes the number periods between 2 dates in relation to the compounding frequency
            :param start_dt: Beginning of period
            :param end_dt: end of period
            :return int: The number of 2 week periods between 2 dates
            """
        if type(start_dt) == str:
            start_dt = date_from_string_to_dt(start_dt)
            end_dt = date_from_string_to_dt(end_dt)

        if self.interest_rate_compounding_period_per_year == 26:
            monday1 = (start_dt - timedelta(days=start_dt.weekday()))
            monday2 = (end_dt - timedelta(days=end_dt.weekday()))
            number_of_interest_rate_compounding_periods = (monday2 - monday1).days / 14

        elif self.interest_rate_compounding_period_per_year == 12:
            number_of_interest_rate_compounding_periods = relativedelta(end_dt, start_dt).years * 12 + relativedelta(end_dt, start_dt).months
        else:
            number_of_interest_rate_compounding_periods = None
            print('Not implemented for compounding frequency', self.interest_rate_compounding_period_per_year)

        return number_of_interest_rate_compounding_periods

    def get_pmt_nominal_value(self, number_of_compounding_period):
        return round(number_of_compounding_period * self.pmt, 2)

    def get_balance_in(self, number_of_compounding_period: str) -> float:
        """
        returns the value of the balance as computed by the difference between the principal value
        accumulated at time period t L*(1=i)**t  and the accumulated value P*Sn

        :param str number_of_compounding_period: Period at which to get the balance
        :return float:

        """
        if type(number_of_compounding_period) is str:

            number_of_compounding_period = self.get_number_of_period_interest_rate_compounding_between_2_dates(self.amortisation_period_start_date_dt,
                                                                                                               date_from_string_to_dt(number_of_compounding_period))
        principal_value: float = self.principal_pv * (1 + self.interest_rate_effective_rate_per_period) ** number_of_compounding_period
        pmt_value = self.pmt * self.get_accumulation_factor_for(number_of_compounding_period)

        return round(principal_value - pmt_value, 4)

    def get_interest_pmt(self, number_of_compounding_period):
        return round(self.get_balance_in(number_of_compounding_period) * self.interest_rate_effective_rate_per_period,
                     2)

    def get_capital_pmt(self, number_of_compounding_period):
        return round(self.pmt - self.get_interest_pmt(number_of_compounding_period), 2)

    def get_repaid_principal(self, number_of_compounding_period):
        return self.principal_pv - self.get_balance_in(number_of_compounding_period)

    def get_amortization_schedule(self, number_of_compounding_period):
        if type(number_of_compounding_period) is str:
            assert self.amortisation_period_start_date_dt is not None, 'To use a string date for the number of period, start_date must be set'
            number_of_compounding_period = self.get_number_of_period_interest_rate_compounding_between_2_dates(
                self.amortisation_period_start_date_dt, date_from_string_to_dt(number_of_compounding_period))

        loan_schedule_dict = {}
        for period in list(range(1, number_of_compounding_period + 1)):
            loan_balance = round(self.principal_pv * (1 + self.interest_rate_effective_rate_per_period) ** (
                        period - 1) - self.pmt * self.get_accumulation_factor_for(period - 1), 4)
            interest_paid = round(loan_balance * self.interest_rate_effective_rate_per_period, 4)
            loan_schedule_dict[period] = (loan_balance,
                                          round(self.pmt, 4),
                                          interest_paid,
                                          round(self.pmt - interest_paid, 4),
                                          round(loan_balance * (
                                                      1 + self.interest_rate_effective_rate_per_period) - self.pmt, 4))
        return loan_schedule_dict

    def plot_amortization_schedule(self, number_of_compounding_period):
        # get loan info in plottable format
        period_list = []
        interest_pmt_list = []
        principal_pm_list = []
        loan_principal = []
        interest_list = []
        loan_balance = []
        for k, period in self.get_amortization_schedule(number_of_compounding_period).items():
            period_list.append(k)
            interest_pmt_list.append(period[1])
            principal_pm_list.append(period[2])
            interest_list.append(period[3])
            loan_principal.append(-period[4])
            loan_balance.append(period[5])

        # get impact on total interest paid and monthly pmt on loan on different interest rate
        delta_i_list = [-0.02, -0.01, -0.005, -0.002, 0, 0.002, 0.005, 0.01, 0.02]
        # create loan for each interest rate variation
        interest_simulation_data_list = []
        for i in delta_i_list:
            #             print(self.interest_rate_annual_nominal_rate, i)
            y = self.interest_rate_annual_nominal_rate + i  # s'assurer que i > 0
            loan = FinancialFunctions(self.amortisation_period_total_number_of_years, self.interest_rate_compounding_period_per_year,
                                      y, self.principal_pv)
            pmt = loan.pmt
            loan_schedule_dict = loan.get_amortization_schedule(
                loan.amortisation_period_total_number_of_years * loan.interest_rate_compounding_period_per_year)
            loan_schedule_list = list(loan_schedule_dict.items())
            loan_schedule_last_element = loan_schedule_list[-1]  # (key, ())
            loan_schedule_last_element_interest_paid = loan_schedule_last_element[1][3]
            interest_simulation_data_list.append((y, pmt, loan_schedule_last_element_interest_paid))
        #
        fig2 = plt.figure(figsize=(16, 8), constrained_layout=True)
        spec2 = GridSpec(ncols=2, nrows=2, figure=fig2)
        f2_ax1 = fig2.add_subplot(spec2[0, 0])
        f2_ax2 = fig2.add_subplot(spec2[0, 1])
        f2_ax3 = fig2.add_subplot(spec2[1, 0])
        f2_ax4 = fig2.add_subplot(spec2[1, 1])

        f2_ax1.bar(period_list, principal_pm_list)
        f2_ax1.bar(period_list, interest_pmt_list, bottom=principal_pm_list)
        f2_ax1.title.set_text('Capiatal vs Interest paiements / paiement with pmt @ {pmt:.2f}'.format(pmt=self.pmt))

        f2_ax2.pie([self.principal_pv, interest_list[-1]], labels=['Principal', 'Interest'], startangle=90)
        f2_ax2.legend()
        f2_ax2.title.set_text(
            'Proportion of Capital ({Capital:.2f}) and interest ({total_interest:.2f}) '
            'paid over loan life @ {interest:.4f} % interest rate'.format(Capital=self.principal_pv,
                                                                          total_interest=interest_list[-1],
                                                                          interest=self.interest_rate_annual_nominal_rate)
        )  # change to self

        f2_ax3.bar(period_list, loan_balance)
        f2_ax3.title.set_text('Loan Balance @ {interest:.4f} % interest rate'.format(
            interest=self.interest_rate_annual_nominal_rate))  # change to self

        for u in list(range(0, len(interest_simulation_data_list))):
            if u != 4:
                colour = '#1f77b4'
            else:
                colour = '#ff7f0e'
            f2_ax4.bar(str(round(interest_simulation_data_list[u][0], 4)), interest_simulation_data_list[u][2],
                       width=0.5, color=[colour])
        f2_ax4.title.set_text('Total interest paid under different interest rate assumptions')
        plot_list = [f2_ax1, f2_ax2, f2_ax3, f2_ax4]
        for plot in plot_list:
            plot.spines['right'].set_visible(False)
            plot.spines['left'].set_visible(False)
            plot.spines['top'].set_visible(False)
            plot.spines['bottom'].set_visible(False)

            plot.set_axisbelow(True)
            plot.yaxis.grid(color='gray', linestyle='dashed', alpha=0.7)


class Loan(FinancialFunctions):
    def __init__(self, contract_period_in_years, amortization_periods_in_years, compounding_period_per_year,
                 interest_rate_annual_nominal_rate,
                 principal_pv, **kwargs):
        """

        :type contract_period_in_years: int
        :param contract_period_in_years:
        :param amortization_periods_in_years:
        :param compounding_period_per_year:
        :param interest_rate_annual_nominal_rate:
        :param principal_pv:
        :param kwargs:
        """
        super().__init__(amortization_periods_in_years, compounding_period_per_year, interest_rate_annual_nominal_rate,
                         principal_pv, pmt=None, **kwargs)
        self.loan_amortisation_period_in_years = contract_period_in_years
        self.loan_amortisation_period_interest_compounding_period = contract_period_in_years * compounding_period_per_year
        if 'start_date' in kwargs:
            self.loan_end_date_dt = self.amortisation_period_start_date_dt + timedelta(
                days=365.25 * self.loan_amortisation_period_in_years)

    def get_loan_value_end_of_contract(self):
        return list(self.get_amortization_schedule(
            self.loan_amortisation_period_in_years * self.interest_rate_compounding_period_per_year).items())[-1][1][5]

    def make_prepayment(self, prepayment_period_number, prepayment_amount):

        # transform from date string to number of period and find the compounding period it corresponds to
        if type(prepayment_period_number) == str:
            self.prepayment_date_dt = date_from_string_to_dt(prepayment_period_number)
            self.prepayment_period_number = self.get_number_of_period_interest_rate_compounding_between_2_dates(
                self.amortisation_period_start_date_dt, self.prepayment_date_dt)
        else:
            self.prepayment_period_number = prepayment_period_number

        # find the balance at the period at which the payment is made and substract it
        self.new_balance_after_prepayment = self.get_balance_in(self.prepayment_period_number) - prepayment_amount

        # find the number of REMAINING period and the number of year for the amortization
        number_of_compounding_period_remaining_to_amortization_period = self.amortisation_total_number_of_interest_compounding_periods - self.prepayment_period_number
        number_of_years_remaining_to_amortization_period: float = number_of_compounding_period_remaining_to_amortization_period / self.interest_rate_compounding_period_per_year

        # find the number of REMAINING period and the number of years until the end of the contract
        number_of_compounding_period_remaining_to_mortgage = self.loan_amortisation_period_in_years * self.interest_rate_compounding_period_per_year - self.prepayment_period_number
        number_of_years_remaining_to_mortgage = number_of_compounding_period_remaining_to_mortgage / self.interest_rate_compounding_period_per_year

        # create new loan based on the new balance and remaining contract and amortization periods
        new_loan = Loan(number_of_years_remaining_to_mortgage,
                        number_of_years_remaining_to_amortization_period,
                        self.interest_rate_compounding_period_per_year,
                        self.interest_rate_annual_nominal_rate,
                        self.new_balance_after_prepayment)

        #         # get the total amount of interest and principal paid up to that point when the pre-paiement is made
        #         interest_principal = list(a.get_amortization_schedule(a.prepayment_period_number).items())[-1][1][3:5]

        # compute the amortisation schedule onward where we add previous interest and principal
        new_ending = {k + self.prepayment_period_number: v
                      for (k, v) in new_loan.get_amortization_schedule(
                number_of_compounding_period_remaining_to_amortization_period).items()}

        # merge the first part of the old
        self.new_amortisation_schedule = self.get_amortization_schedule(self.prepayment_period_number) | new_ending


class House(object):
    def __init__(self, owner, type, purchased_value, cash_down, financing, purchased_date, municipal_assessment=307900,
                 **kwargs):
        """

        :param owner:
        :param str type: The residence type, principal, secondary, etc, the intention is tu use is as tax purposes
        :param int market_value: The market value of the house at purchased
        :param int cash_down: Cash down amount
        :param Loan financing:
        :param str purchased_date:
        :param int municipal_assessment:
        :param kwargs:
        """
        self.owner = owner
        self.type = type
        self.purchased_value = purchased_value
        self.cash_down = cash_down
        self.municipal_assessement = municipal_assessment
        #         self.annual_growth_rate = annual_growth_rate
        self.mortgage = financing

    def get_taxes_municipale(self):
        # https://res.cloudinary.com/villemontreal/image/upload/v1611858609/portail/vjtpg2vpsafmeslabh3k.pdf
        taxes = {'montreal': {'taxes_foncières_générales_5_logements_ou_moins': 0.6117,
                              'Taxe_spéciale_relative_ARTM_5_logements_ou_moins': 0.0023,
                              'Taxe_spéciale_eau': 0.0998,
                              'Taxe_voirie': 0.0033,
                              'Taxe_services': 0.0843,
                              'Taxe_investissements': 0.0725
                              }
                 }
        multiplicator = self.municipal_assessement / 100
        municipal_taxe_name_list = []
        municipal_taxe_amount_list = []
        for city, type_taxes in taxes.items():
            for taxe_name, taxe_rate in type_taxes.items():
                municipal_taxe_name_list.append(taxe_name)
                municipal_taxe_amount_list.append(taxe_rate * multiplicator)

        return (municipal_taxe_name_list, municipal_taxe_amount_list)



In [3]:
# date achat 2018-05-01
# pmt 10 000 2021-02-01

Hyp = Loan(5, 30, 12, 0.0205, 320000, start_date='2018-05-01')
# loan1 = Loan(5,0.05, 20000)

#### New schedule with pre-payment while maintaining pmt constant

In [ ]:
print(principal_pv)

print('remaining period to loan from prepayment', 
      a.get_number_of_period_interest_rate_compounding_between_2_dates(a.prepayment_date_dt, a.amortisation_period_end_date_dt)+1)
new_loan_schedule_dict = {}
for period in list(range(1, a.get_number_of_period_interest_rate_compounding_between_2_dates(a.prepayment_date_dt, 
                                                                                            a.loan_end_date_dt)+1)):
    loan_balance = round(principal_pv * (1 + a.interest_rate_effective_rate_per_period) ** (period-1) - a.pmt * a.get_accumulation_factor_for(period-1), 2)
    interest_paid = round(loan_balance * a.interest_rate_effective_rate_per_period, 2)
    new_loan_schedule_dict[period] = (loan_balance, 
                                      round(a.pmt, 4),
                                      interest_paid,
                                      round(a.pmt - interest_paid, 4),
                                      round(loan_balance* (1 + a.interest_rate_effective_rate_per_period) - a.pmt, 4)) 
    
amortization_schedule_new_ending = {k+a.prepayment_period_number : v for (k,v) in new_loan_schedule_dict.items() if v[4]>0}
# print(amortization_schedule_new_ending)
amortisation_schedule_begining = a.get_amortization_schedule(a.prepayment_period_number)
# print(amortisation_schedule_begining)
total_new_amortization_schedule = amortisation_schedule_begining | amortization_schedule_new_ending

total_amortization_schedule

amortization_last_period = list(total_amortization_schedule.items())[-1][0]
amortization_last_period
# [(total_new_amortization_schedule[k][2],
#   a.get_amortization_schedule(a.number_of_compounding_periods)[k][2],
#   round(total_new_amortization_schedule[k][2] - a.get_amortization_schedule(a.number_of_compounding_periods)[k][2],2))
#      for k in list(range(1, a.contract_end_period+1))
# ]

# interest_saved = sum([round(total_new_amortization_schedule[k][2] - a.get_amortization_schedule(a.number_of_compounding_periods)[k][2],2)
#      for k in list(range(1, a.amortisation_periods_in_years*a.compounding_period_per_year-1))
# ])

# interest_saved



In [ ]:
[sum(v[2] for (k,v) in new_loan.get_amortization_schedule(a.contract_period_in_years * a.compounding_period_per_year - a.prepayment_period_number).items())]

In [ ]:
loan_schedule_dict = {}
for period in list(range(1, a.number_of_compounding_periods+1)):
    loan_balance = round(a.principal_pv * (1 + a.effective_interest_rate_per_period) ** (period-1) - a.pmt * a.get_accumulation_factor_for(period-1), 2)
    interest_paid = round(loan_balance*a.effective_interest_rate_per_period,2)
    loan_schedule_dict[period] = (loan_balance, 
                                  round(a.pmt, 4),
                                  interest_paid,
                                  round(a.pmt - interest_paid, 4),
                                  round(loan_balance * (1 + a.effective_interest_rate_per_period) - a.pmt, 4)) 
    

In [ ]:
round(sum([loan_schedule_dict[k][2] for k in list(range(1,a.number_of_compounding_periods+1))]),2)

In [ ]:
round(sum([loan_schedule_dict[k][3] for k in list(range(1,a.number_of_compounding_periods+1))]),2)